In [1]:
!pip install keras

In [2]:
!pip install tensorflow

In [3]:
!pip install opencv-python

In [4]:
!pip install imutils

In [5]:
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from keras.models import Model, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np

In [6]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
import os

In [7]:
# Define model architecture
model = Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(2, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])


In [8]:
# Define training and validation data generators
TRAINING_DIR = "./data"
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size=10, 
                                                    target_size=(150, 150))

VALIDATION_DIR = "./data"
validation_datagen = ImageDataGenerator(rescale=1.0/255)

validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                              batch_size=10, 
                                                              target_size=(150, 150))

Found 305 images belonging to 2 classes.
Found 305 images belonging to 2 classes.


In [9]:
# Callbacks
checkpoint = ModelCheckpoint('model2-{epoch:03d}.model', monitor='val_loss', verbose=0, save_best_only=True, mode='auto')
early_stopping = EarlyStopping(monitor='val_loss', patience=5)


In [10]:
# Calculate class weights for imbalanced datasets
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

In [11]:
classes = train_generator.classes
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(classes), y=classes)
class_weights = dict(enumerate(class_weights))

In [ ]:
# Train the model
history = model.fit(train_generator,
                    epochs=10,
                    validation_data=validation_generator,
                    callbacks=[checkpoint, early_stopping],
                    class_weight=class_weights)  # Adjust for class imbalance


Epoch 1/10
31/31 [==============================] - ETA: 0s - loss: 0.8007 - acc: 0.5574INFO:tensorflow:Assets written to: model2-001.model/assets


INFO:tensorflow:Assets written to: model2-001.model/assets


31/31 [==============================] - 9s 278ms/step - loss: 0.8007 - acc: 0.5574 - val_loss: 0.6796 - val_acc: 0.7443
Epoch 2/10
31/31 [==============================] - ETA: 0s - loss: 0.6357 - acc: 0.6525INFO:tensorflow:Assets written to: model2-002.model/assets


INFO:tensorflow:Assets written to: model2-002.model/assets


31/31 [==============================] - 8s 271ms/step - loss: 0.6357 - acc: 0.6525 - val_loss: 0.6449 - val_acc: 0.6131
Epoch 3/10
31/31 [==============================] - ETA: 0s - loss: 0.6204 - acc: 0.6393INFO:tensorflow:Assets written to: model2-003.model/assets


...

31/31 [==============================] - 8s 261ms/step - loss: 0.5186 - acc: 0.7508 - val_loss: 0.4624 - val_acc: 0.7639
Epoch 7/10
31/31 [==============================] - ETA: 0s - loss: 0.4904 - acc: 0.7475INFO:tensorflow:Assets written to: model2-007.model/assets


INFO:tensorflow:Assets written to: model2-007.model/assets


31/31 [==============================] - 8s 264ms/step - loss: 0.4904 - acc: 0.7475 - val_loss: 0.4421 - val_acc: 0.7803
Epoch 8/10
31/31 [==============================] - ETA: 0s - loss: 0.4670 - acc: 0.7770INFO:tensorflow:Assets written to: model2-008.model/assets


INFO:tensorflow:Assets written to: model2-008.model/assets


31/31 [==============================] - 9s 286ms/step - loss: 0.4670 - acc: 0.7770 - val_loss: 0.3936 - val_acc: 0.8098
Epoch 9/10
31/31 [==============================] - ETA: 0s - loss: 0.4815 - acc: 0.7836

In [ ]:
model.save('my_model.keras')  # creates a HDF5 file 'my_model.h5'
del model
